In [2]:
#File name: FullSongTest
#Names of all members of the group: James Kaufman, Eamon Kostopulos, 
#Ahmed Mohammed, Sebastian Cortes, and Jonathan Goral 
#Project name and description: DL Music Classification - A neural network that classifies music genres
#Any special execution instruction: librosa, numpy, pydub, os, pickle, pylab, ffmeg, glob, matplotlib.pyplot
#Date: 12/4/18

import librosa
import librosa.feature
import os
import numpy as np
from keras.models import load_model
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.utils.np_utils import to_categorical
from keras.preprocessing import sequence
from keras import initializers
import pickle


Using TensorFlow backend.


In [4]:
#load in our model and the saved mapping we have
model = load_model("5genreweights.h5")
with open ("mapping.pkl", "rb") as f:
    newMap = pickle.load(f)

#we need to reverse the mapping because it is saved in the format of
#GenreName : Output Integer
#whereas we want it in the format
#Output Integer : GenreName
genre_decode = {v: k for k, v in newMap.items()}

(128, 8189)


In [6]:
#this function takes in a song, converts it into a spectrogram, and splits that spectrogram
#into 5 second long snippets.
def splitFile(filename):
        #load in the song and convert it to a spectrogram
        song, sr = librosa.load(filename, mono=True, sr = 22050)
        mel = librosa.feature.melspectrogram(song, sr)
        max_len = 217
        #r is how many different snippets we will end up with: it divides the total length
        #of the spectrogram by the length that represents a five second snippet
        #and then adds one to round up since integer conversions round down.
        r = int(mel.shape[1]/max_len) + 1
        specs = []
        for i in range(r):
            temp = mel[:,i*217:(i+1)*217]
            #this is neccessary in case the song does not end cleanly, as it will pad
            #the last snippet of the song with zeroes.
            if (max_len > temp.shape[1]):
                pad_width = max_len - temp.shape[1]
                temp = np.pad(temp, pad_width=((0, 0), (0, pad_width)), mode='constant')
            #reshape this slice to be in a usable format for our neural network
            temp= temp.reshape(1,temp.shape[0],temp.shape[1],1)
            specs.append(temp)
        return specs

In [13]:
#This method takes care of guessing what genre a song is if given a list of snippets.
#It does so with a voting mechanism - the neural network takes in each snippet and 
#classifies it as a certain genre.  We store how many votes each genre gets and then 
#return the genre that had the most votes as the genre that the song will be classified as
def guessSong(splitSong):
    #creates the dictionary we wil use to store our guesses
    guesses = {}
    for part in splitSong:
        #take the 0th element since predict_proba returns a list with a list inside of it
        probs = model.predict_proba(part)[0]
        confidence = 0
        #predict_proba returns a list of the how confident it would be in classifying
        #the snippet as any one of the five genres.  we iterate through all of these
        #and pick the maximum confidence it has for any genre
        for prob in probs:
            if prob > confidence:
                confidence = prob
        #throw out the vote if it is not more than 50% sure of any genre, because
        #the vote is meaningless at that point
        if confidence < .5:
            x = 1
        else:
            #get the actual prediction for the genre
            guess = model.predict_classes(part)
            genre_name = genre_decode[guess[0]]
            #update the amount of votes that genre has
            if genre_name in guesses:
                guesses[genre_name] = guesses[genre_name] + 1
            else:
                guesses[genre_name] = 1
    max_guesses = 0
    prediction = ""
    #get our actual prediction.  go through the entire dictionary of how many votes each
    #genre got and find which one got the most, and store the name of that genre in prediction
    for key in guesses:
        if guesses[key] > max_guesses:
            prediction = key
            max_guesses = guesses[key]
    return prediction

In [1]:
#This takes care of testing the neural network on a lot of data at once - we have numerous
#folders which are all named "(GenreName)Test", so it goes into each folder, classifies all
#the songs in there, and tracks how accurate it was for that genre as well as how accurate
#it is overall

folders = os.listdir("Database/Test Data/")

overall_right = 0
overall_songs = 0
wrongGuesses = []

for folder in folders:
    
    files = os.listdir("Database/Test Data/" + folder + "/")
    numFiles = len(files)
    #since each folder is named (GenreName)Test, this removes the word "Test" from the folder name
    genreName = folder[:-4]
    rightGuesses = 0
    for filename in files:
        #use our methods we already created to guess what genre this song is
        split = splitFile("Database/Test Data/" + folder + "/" + filename)
        prediction = guessSong(split)
        
        overall_songs = overall_songs+1
        #if our prediction for this song matched the genre name from the folder name, we
        #were right.  Otherwise store that song name as well as the genre the neural
        #network thought it was so we can see all the wrong guesses later.
        if prediction == genreName:
            rightGuesses = rightGuesses+1
            overall_right = overall_right + 1
        else:
            wrongGuesses.append("I thought " + filename + " was " + prediction + " but its " + genreName)
    accuracy = rightGuesses/numFiles * 100
    print("The model was " + str(accuracy) + "% accurate at classifying " + genreName + " songs")
    
overall_acc = overall_right/overall_songs
print("It was " + str(overall_acc) + "% accurate in general.")

print("Here is what I was wrong about:")
for string in wrongGuesses:
    print(string)

NameError: name 'os' is not defined

In [2]:
#print out every wrong guess that we had so we can see what the network is struggling with
for string in wrongGuesses:
    print(string)

NameError: name 'wrongGuesses' is not defined